# YOLO
YOLO (You Only Look Once) é um sistema construído com base em uma rede neural convolucional, cuja arquitetura é chamada de Darknet, que é o mesmo nome do framework utilizado para implementar o detector. O sistema é capaz de identificar e localizar objetos em uma imagem ou vídeo, podendo funcionar em GPU ou CPU. Foi utilizado a versão 4 do YOLO, adaptando-o para detectar placas de veículos na etapa de implementação da rede.

# Configurando a biblioteca Pytesseract


In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract

In [ ]:
import os

os.environ['GPU_TYPE'] = str(os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read())

print("GPU: " + os.environ['GPU_TYPE'])

# Instalando o Darknet




In [ ]:
%cd /content/
%rm -rf darknet

In [ ]:
#Clonando o Darknet para o Colab
!git clone https://github.com/AlexeyAB/darknet
%mv /content/y* /content/darknet/cfg

In [ ]:
%cd /content/darknet/

In [ ]:
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!chmod +x *.sh

In [ ]:
!make

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cp '/content/drive/MyDrive/ProfJefferson/Rede Neural/custom-yolov4-detector_final.weights' '/content/darknet/backup/custom-yolov4-detector_final.weights'
%cp '/content/drive/MyDrive/ProfJefferson/Rede Neural/Config/custom-yolov4-detector.cfg' '/content/darknet/cfg/custom-yolov4-detector.cfg'
%cp '/content/drive/MyDrive/ProfJefferson/Rede Neural/Config/obj.names' '/content/darknet/data/obj.names'
%cp '/content/drive/MyDrive/ProfJefferson/Rede Neural/Config/obj.data' '/content/darknet/data/classes.data'
%cp '/content/drive/MyDrive/ProfJefferson/Python OpenCV e Pytesseract/br.xml' '/content/br.xml'

%cp -r '/content/drive/MyDrive/ProfJefferson/images' '/content/images'

In [ ]:
%cd /content/darknet/

In [ ]:
import darknet
import darknet_images

CFG = "/content/darknet/cfg/custom-yolov4-detector.cfg"
PESO = "/content/darknet/backup/custom-yolov4-detector_final.weights"
DATA = "/content/darknet/data/classes.data"
NAMES = "/content/darknet/data/obj.names"
network, name, colors = darknet.load_network(CFG,DATA,PESO)

In [ ]:
def bbox2points(bbox):
    x, y, w, h = bbox
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
import pytesseract

imagemInput = cv2.imread("/content/images/BAJ-4441.jpg")

imagemInput = cv2.cvtColor(imagemInput, cv2.COLOR_BGR2GRAY)
imagemInput = cv2.GaussianBlur(imagemInput, (9,9), 0)
ret3,imagemInput = cv2.threshold(imagemInput,0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
kernel = np.ones((5,5),np.uint8)

imagemInput1 = cv2.dilate(imagemInput, kernel, iterations = 1)
closing = cv2.morphologyEx(imagemInput, cv2.MORPH_CLOSE, rect_kern)
opening = cv2.morphologyEx(imagemInput, cv2.MORPH_OPEN, rect_kern)

Text = pytesseract.image_to_string(imagemInput1, lang='eng',config='--psm 6 --oem 1')
print(Text + " Dilate!")
cv2_imshow(imagemInput)
Text = pytesseract.image_to_string(closing, lang='eng',config='--psm 6 --oem 1')
print(Text + " Close")
cv2_imshow(imagemInput)


In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
# import numpy as np
# import darknet
# import darknet_images
from google.colab.patches import cv2_imshow
import pytesseract

placasXml = '/content/br.xml'
placasCascade = cv2.CascadeClassifier(placasXml)

path = "/content/images"
dirs = os.listdir( path )

for item in dirs:
    teste = False
    imagemInput = cv2.imread("/content/images/"+item)

    imagemInput = cv2.cvtColor(imagemInput, cv2.COLOR_BGR2GRAY)
    imagemInput = cv2.GaussianBlur(imagemInput, (5,5), 0)
    ret3,imagemInput = cv2.threshold(imagemInput,0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    # imagemInput = cv2.dilate(imagemInput, rect_kern, iterations = 1)
    kernel = np.ones((5,5),np.uint8)

    opening = cv2.morphologyEx(imagemInput, cv2.MORPH_OPEN, kernel)


    placas = placasCascade.detectMultiScale(
        imagemInput,
        scaleFactor=1.05,
        minNeighbors=3,
        minSize=(30, 30)
    )

    for (x, y, w, h) in placas:
            recortePlaca = imagemInput[y:y + h, x:x + w]

            Text = pytesseract.image_to_string(recortePlaca, lang='eng',config='--psm 6 --oem 1')
            print(Text + " foi o texto achado na placa!")
# cv2_imshow(imagemInput)


# for label, confidence, bbox in detected:
#   left, top, right, bottom = bbox2points(bbox)
#   imgRecortePlaca = Image1.crop((left, top, right, bottom))

#   alturaRecortePlaca, larguraRecortePlaca = Image1.size
#   print(alturaRecortePlaca, larguraRecortePlaca)
#   larguraRecortePlaca = round(larguraRecortePlaca * proporcao)
#   alturaRecortePlaca = round(alturaRecortePlaca * 0.8)
#   imgRecortePlaca = imgRecortePlaca.resize((larguraRecortePlaca, alturaRecortePlaca))

#   Text = pytesseract.image_to_string(imgRecortePlaca,lang='eng',config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
#   print(Text)
#   display(imgRecortePlaca)



In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pytesseract
import os


placasXml = '/content/br.xml'
placasCascade = cv2.CascadeClassifier(placasXml)

path = "/content/images"
dirs = os.listdir( path )

acertos = 0
erros = 0
contTotal = 0
cont = 0

for item in dirs:
    print(item)
    teste = False
    contTotal += 1
    Text = ""

    imagemInput = cv2.imread("/content/images/"+item)

    nomeArquivoAtual = item.replace('.jpg', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('.JPG', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('-', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('(1)', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('(2)', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('-1', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('-2', '')

    placas = placasCascade.detectMultiScale(
        imagemInput,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(30, 30)
    )

    imagemInput = cv2.cvtColor(imagemInput, cv2.COLOR_BGR2GRAY)
    imagemInput = cv2.GaussianBlur(imagemInput,(9,9),0)
    # ret3,imagemInput = cv2.threshold(imagemInput,0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    # imagemInput = cv2.dilate(imagemInput, rect_kern, iterations = 1)

    for (x, y, w, h) in placas:
      recortePlaca = imagemInput[y:y + h, x:x + w]
      cv2.imwrite('/content/imgRecortadaBala/'+nomeArquivoAtual+'.jpg', recortePlaca)
      cont += 1
      Text = pytesseract.image_to_string(recortePlaca, lang='eng',config='--psm 6 --oem 1')
      Text = Text.replace('~', '')
      Text = Text.replace('-', '')
      Text = Text.replace(':', '')
      Text = Text.replace('!', '')
      Text = Text.replace(' ', '')
      Text = Text.replace('`', '')
      Text = Text.replace('|', '')
      Text = Text.replace('.', '')
      Text = Text.replace('‘', '')
      Text = Text.replace('/', '')
      Text = Text.replace('>', '')
      Text = Text.replace('<', '')
      Text = Text.replace(' ', '')
      Text = Text.replace('@', '')
      Text = Text.replace("'", '')


      if len(Text)-2 == 7:
        letrasPlaca = Text[:3]
        numerosPlaca = Text[3:7]
        letrasPlaca = letrasPlaca.replace('0', 'O')
        letrasPlaca = letrasPlaca.replace('1', 'I')
        letrasPlaca = letrasPlaca.replace('6', 'G')
        letrasPlaca = letrasPlaca.replace('4', 'A')
        letrasPlaca = letrasPlaca.replace('5', 'S')
        letrasPlaca = letrasPlaca.replace('[', 'I')

        numerosPlaca = numerosPlaca.replace('I', '1')
        numerosPlaca = numerosPlaca.replace('S', '5')
        numerosPlaca = numerosPlaca.replace('G', '6')
        numerosPlaca = numerosPlaca.replace('A', '4')
        numerosPlaca = numerosPlaca.replace('S', '5')
        numerosPlaca = numerosPlaca.replace('S', '5')
        numerosPlaca = numerosPlaca.replace('[', '1')
        Text = letrasPlaca+numerosPlaca
      print(Text + " foi o texto achado na placa:"+nomeArquivoAtual)

      if(Text.find(nomeArquivoAtual) > -1):
          print("ACERTOU")
          acertos += 1
          break
      else:
          erros += 1

print("Saiu do loop")
TaxaAcerto = acertos/cont*100
TaxaErro = erros/cont*100

contTotalText = str(contTotal)
contText = str(cont)
TaxaAcerto = str(TaxaAcerto)
TaxaErro = str(TaxaErro)
print("Total de arquivos percorridos:" + contTotalText +"\nTotal de placas encontradas:" + contText +
"\nTotal de acertos: " + TaxaAcerto +
"%\nTotal de erros: " + TaxaErro+"%")
cv2.waitKey(0)



In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pytesseract
import os
import darknet
import darknet_images


path = "/content/images/"
dirs = os.listdir( path )

acertos = 0
erros = 0
contTotal = 0
cont = 0

for item in dirs:
    print(item)
    cont += 1
    teste = False
    contTotal += 1
    Text = ""
    imagemInput = cv2.imread(path+item)

    alturaImagemInput, larguraImagemInput = imagemInput.shape[:2]
    proporcao = larguraImagemInput/alturaImagemInput;
    proporcao = round(proporcao, 2)

    nomeArquivoAtual = item.replace('.jpg', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('.JPG', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('-', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('(1)', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('(2)', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('-1', '')
    nomeArquivoAtual = nomeArquivoAtual.replace('-2', '')

    imagemInput , detected = darknet_images.image_detection(imagemInput, network, name,colors, thresh=0.5)
    imagemInput = cv2.cvtColor(imagemInput, cv2.COLOR_BGR2GRAY)

    imagemInput = cv2.GaussianBlur(imagemInput, (7,7),0)

    ret3,imagemInput = cv2.threshold(imagemInput,0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    kernel = np.ones((5,5),np.uint8)
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    imagemInput = cv2.dilate(imagemInput, kernel, iterations = 1)

    cv2.imwrite('imgDetectada.jpg', imagemInput)

    Image1 = Image.open("/content/darknet/imgDetectada.jpg")

    for label, confidence, bbox in detected:
      left, top, right, bottom = bbox2points(bbox)
      imgRecortePlaca = Image1.crop((left, top, right, bottom))
      alturaRecortePlaca, larguraRecortePlaca = Image1.size
      larguraRecortePlaca = round(larguraRecortePlaca * proporcao*0.7)
      alturaRecortePlaca = round(alturaRecortePlaca * 0.4)
      imgRecortePlaca = imgRecortePlaca.resize((larguraRecortePlaca, alturaRecortePlaca))

      Text = pytesseract.image_to_string(imgRecortePlaca,lang='eng',config='--psm 6 --oem 1')
      Text = Text.replace('~', '')
      Text = Text.replace('-', '')
      Text = Text.replace(':', '')
      Text = Text.replace('!', '')
      Text = Text.replace(' ', '')
      Text = Text.replace('`', '')
      Text = Text.replace('|', '')
      Text = Text.replace('.', '')
      Text = Text.replace('‘', '')
      Text = Text.replace('/', '')
      Text = Text.replace('>', '')
      Text = Text.replace('<', '')
      Text = Text.replace(' ', '')
      Text = Text.replace('@', '')
      Text = Text.replace("'", '')


      if len(Text)-2 == 7:
        letrasPlaca = Text[:3]
        numerosPlaca = Text[3:7]
        letrasPlaca = letrasPlaca.replace('0', 'O')
        letrasPlaca = letrasPlaca.replace('1', 'I')
        letrasPlaca = letrasPlaca.replace('6', 'G')
        letrasPlaca = letrasPlaca.replace('4', 'A')
        letrasPlaca = letrasPlaca.replace('5', 'S')
        letrasPlaca = letrasPlaca.replace('[', 'I')

        numerosPlaca = numerosPlaca.replace('I', '1')
        numerosPlaca = numerosPlaca.replace('S', '5')
        numerosPlaca = numerosPlaca.replace('G', '6')
        numerosPlaca = numerosPlaca.replace('A', '4')
        numerosPlaca = numerosPlaca.replace('S', '5')
        numerosPlaca = numerosPlaca.replace('S', '5')
        numerosPlaca = numerosPlaca.replace('[', '1')
        Text = letrasPlaca+numerosPlaca
      print(Text + " foi o texto achado na placa:"+nomeArquivoAtual)

      if(Text.find(nomeArquivoAtual) > -1):
          print("ACERTOU")
          acertos += 1
          break
      else:
          erros += 1

print("Saiu do loop")
TaxaAcerto = acertos/cont*100
TaxaErro = erros/cont*100

acertosText = str(acertos)

contTotalText = str(contTotal)
contText = str(cont)
TaxaAcerto = str(TaxaAcerto)
TaxaErro = str(TaxaErro)
print("Total de arquivos percorridos:" + contTotalText +"\nTotal de placas encontradas:" + contText +
"\nTotal de acertos: " + TaxaAcerto + acertosText +
"%\nTotal de erros: " + TaxaErro+"%")
cv2.waitKey(0)

